In [1]:
import numpy as np
class CustomDecisionTree:
  def __init__(self, max_depth=None):
    """
    Initializes the decision tree with the specified maximum depth.
    Parameters:
    max_depth (int, optional): The maximum depth of the tree. If None, the tree is expanded until all
    leaves are pure or contain fewer than the minimum samples required to split.
    """
    self.max_depth = max_depth
    self.tree = None

  def fit(self, X, y):
    """
    Trains the decision tree model using the provided training data.
    Parameters:
    X (array-like): Feature matrix (n_samples, n_features) for training the model.
    y (array-like): Target labels (n_samples,) for training the model.
    """
    self.tree = self._build_tree(X, y)

  def _build_tree(self, X, y, depth=0):
        """
        Recursively builds the decision tree by splitting the data based on the best feature and threshold
        .
        Parameters:
        X (array-like): Feature matrix (n_samples, n_features) for splitting.
        y (array-like): Target labels (n_samples,) for splitting.
        depth (int, optional): Current depth of the tree during recursion.
        Returns:
        dict: A dictionary representing the structure of the tree, containing the best feature index,
        threshold, and recursive tree nodes.
        """
        num_samples, num_features = X.shape
        unique_classes = np.unique(y)
        # Stopping conditions: pure node or reached max depth
        if len(unique_classes) == 1:
            return {'class': unique_classes[0]}
        if num_samples == 0 or (self.max_depth and depth >= self.max_depth):
            return {'class': np.bincount(y).argmax()}
        # Find the best split based on Information Gain (using Entropy)
        best_info_gain = -float('inf')
        best_split = None
        for feature_idx in range(num_features):
            thresholds = np.unique(X[:, feature_idx])
            for threshold in thresholds:
                left_mask = X[:, feature_idx] <= threshold
                right_mask = ~left_mask
                left_y = y[left_mask]
                right_y = y[right_mask]
                info_gain = self._information_gain(y, left_y, right_y)
                if info_gain > best_info_gain:
                    best_info_gain = info_gain
                    best_split = {
                        'feature_idx': feature_idx,
                        'threshold': threshold,
                        'left_mask': left_mask,  # Store the mask instead of y values
                        'right_mask': right_mask,  # Store the mask instead of y values
                    }

        if best_split is None:
            return {'class': np.bincount(y).argmax()}
        # Recursively build the left and right subtrees
        # Use the masks to subset X and y for the recursive calls
        left_tree = self._build_tree(X[best_split['left_mask']], y[best_split['left_mask']], depth + 1)
        right_tree = self._build_tree(X[best_split['right_mask']], y[best_split['right_mask']], depth + 1)
        return {'feature_idx': best_split['feature_idx'], 'threshold': best_split['threshold'], 'left_tree': left_tree, 'right_tree': right_tree}

  def _information_gain(self, parent, left, right):
    """
    Computes the Information Gain between the parent node and the left/right child nodes.
    Parameters:
    parent (array-like): The labels of the parent node.
    left (array-like): The labels of the left child node.
    right (array-like): The labels of the right child node.
    Returns:
    float: The Information Gain of the split.
    """
    parent_entropy = self._entropy(parent)
    left_entropy = self._entropy(left)
    right_entropy = self._entropy(right)
    # Information Gain = Entropy(parent) - (weighted average of left and right entropies)
    weighted_avg_entropy = (len(left) / len(parent)) * left_entropy + (len(right) / len(parent)) *right_entropy
    return parent_entropy - weighted_avg_entropy

  def _entropy(self, y):
    """
    Computes the entropy of a set of labels.
    Parameters:
    y (array-like): The labels for which entropy is calculated.
    Returns:
    float: The entropy of the labels.
    """
    # Calculate the probability of each class
    class_probs = np.bincount(y) / len(y)
    # Compute the entropy using the formula: -sum(p * log2(p))
    return -np.sum(class_probs * np.log2(class_probs + 1e-9)) # Added small epsilon to avoid log(0)

  def predict(self, X):
    """
    Predicts the target labels for the given test data based on the trained decision tree.
    Parameters:
    X (array-like): Feature matrix (n_samples, n_features) for prediction.
    Returns:
    list: A list of predicted target labels (n_samples,).
    """
    return [self._predict_single(x, self.tree) for x in X]

  def _predict_single(self, x, tree):
    """
    Recursively predicts the target label for a single sample by traversing the tree.
    Parameters:
    x (array-like): A single feature vector for prediction.
    tree (dict): The current subtree or node to evaluate.
    Returns:
    int: The predicted class label for the sample.
    """
    if 'class' in tree:
      return tree['class']
    feature_val = x[tree['feature_idx']]
    if feature_val <= tree['threshold']:
      return self._predict_single(x, tree['left_tree'])
    else:
      return self._predict_single(x, tree['right_tree'])

In [2]:
# Necessary Imports
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
# Load the Iris dataset
data = load_iris()
X = data.data
y = data.target
# Split into training and test sets (80% training, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Train the custom decision tree
custom_tree = CustomDecisionTree(max_depth=3)
custom_tree.fit(X_train, y_train)
# Predict on the test set
y_pred_custom = custom_tree.predict(X_test)
# Calculate accuracy
accuracy_custom = accuracy_score(y_test, y_pred_custom)
print(f"Custom Decision Tree Accuracy: {accuracy_custom:.4f}")
# Train the Scikit-learn decision tree
sklearn_tree = DecisionTreeClassifier(max_depth=3, random_state=42)
sklearn_tree.fit(X_train, y_train)
# Predict on the test set
y_pred_sklearn = sklearn_tree.predict(X_test)
# Calculate accuracy
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)
print(f"Scikit-learn Decision Tree Accuracy: {accuracy_sklearn:.4f}")
print(f"Accuracy Comparison:")
print(f"Custom Decision Tree: {accuracy_custom:.4f}")
print(f"Scikit-learn Decision Tree: {accuracy_sklearn:.4f}")

Custom Decision Tree Accuracy: 1.0000
Scikit-learn Decision Tree Accuracy: 1.0000
Accuracy Comparison:
Custom Decision Tree: 1.0000
Scikit-learn Decision Tree: 1.0000


In [3]:

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import f1_score, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

wine = load_wine()
X = wine.data
y = wine.target

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 1. Classification Models
print("Part 1: Classification Models")
print("-" * 50)

# Decision Tree Classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train_scaled, y_train)
dt_pred = dt_clf.predict(X_test_scaled)
dt_f1 = f1_score(y_test, dt_pred, average='weighted')
print(f"Decision Tree F1 Score: {dt_f1:.4f}")

# Random Forest Classifier
rf_clf = RandomForestClassifier(random_state=42)
rf_clf.fit(X_train_scaled, y_train)
rf_pred = rf_clf.predict(X_test_scaled)
rf_f1 = f1_score(y_test, rf_pred, average='weighted')
print(f"Random Forest F1 Score: {rf_f1:.4f}")

# 2. Hyperparameter Tuning for Random Forest Classifier
print("\nPart 2: Hyperparameter Tuning (Random Forest Classifier)")
print("-" * 50)

# Define parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Perform GridSearchCV
rf_clf_grid = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,
    scoring='f1_weighted',
    n_jobs=-1
)

rf_clf_grid.fit(X_train_scaled, y_train)
best_rf_pred = rf_clf_grid.predict(X_test_scaled)
best_rf_f1 = f1_score(y_test, best_rf_pred, average='weighted')

print("Best parameters:", rf_clf_grid.best_params_)
print(f"Best F1 Score: {best_rf_f1:.4f}")

# 3. Regression Models
print("\nPart 3: Regression Models")
print("-" * 50)

# For regression, we'll predict the first feature using the others
X_reg = np.delete(X, 0, axis=1)  # Remove first feature
y_reg = X[:, 0]  # Use first feature as target

# Split data for regression
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

# Scale features
X_train_reg_scaled = scaler.fit_transform(X_train_reg)
X_test_reg_scaled = scaler.transform(X_test_reg)

# Decision Tree Regressor
dt_reg = DecisionTreeRegressor(random_state=42)
dt_reg.fit(X_train_reg_scaled, y_train_reg)
dt_reg_pred = dt_reg.predict(X_test_reg_scaled)
dt_reg_r2 = r2_score(y_test_reg, dt_reg_pred)
print(f"Decision Tree Regressor R2 Score: {dt_reg_r2:.4f}")

# Random Forest Regressor with RandomizedSearchCV
# Define parameter distribution
param_dist = {
    'n_estimators': np.arange(50, 200, 25),
    'max_depth': [None] + list(np.arange(5, 30, 5)),
    'min_samples_leaf': np.arange(1, 5)
}

# Perform RandomizedSearchCV
rf_reg_random = RandomizedSearchCV(
    RandomForestRegressor(random_state=42),
    param_distributions=param_dist,
    n_iter=20,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    random_state=42
)

rf_reg_random.fit(X_train_reg_scaled, y_train_reg)
best_rf_reg_pred = rf_reg_random.predict(X_test_reg_scaled)
best_rf_reg_r2 = r2_score(y_test_reg, best_rf_reg_pred)

print("\nRandom Forest Regressor Results:")
print("Best parameters:", rf_reg_random.best_params_)
print(f"Best R2 Score: {best_rf_reg_r2:.4f}")

Part 1: Classification Models
--------------------------------------------------
Decision Tree F1 Score: 0.9440
Random Forest F1 Score: 1.0000

Part 2: Hyperparameter Tuning (Random Forest Classifier)
--------------------------------------------------
Best parameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 50}
Best F1 Score: 1.0000

Part 3: Regression Models
--------------------------------------------------
Decision Tree Regressor R2 Score: 0.5016

Random Forest Regressor Results:
Best parameters: {'n_estimators': 50, 'min_samples_leaf': 4, 'max_depth': 10}
Best R2 Score: 0.7616
